# Research Paper Midway Point

__Deadline Warrior__: Jitao Zhang, Wanqing Hu, Siying Liu

## 1. Research Summary

This research investigates the impact of diabetes on the health of Californians from 2016 to 2022, focusing on hospitalizations and deaths. It analyzes trends and patterns to understand the burden of diabetes across different counties and complications. The findings aim to inform about the health conditions of Californians and provide insights for better healthcare management.

## 2. Revised Research Questions

### Generial Direction:

We will explore how diabetes has impacted the health of the Californian population from 2016 to 2022, as measured by the number of hospitalizations and deaths.

### Three Pathways:

_Jitao_
- Original Question: The relationship between the number of deaths due to diabetes and the number of hospitalizations due to diabetes in California from 2016 to 2022.
- __Revised Question:__ Is there a relationship between the number of deaths due to diabetes in California with respect to the number of hospitalizations related to diabetes and the at-risk population using data from 2016 to 2022?
- Justification: In my original question, there is essentially one response and just one explanatory variable, which may overlook some lurking or confounding variables' effects as TA suggested. Therefore, this time, i choose to add one more variable, population at risk from all counties of california each year as a second explanatory variable. however, since there is just 7 observations in another dataset after we clean it, we can't 

_Wanqing_
- Original question: The trend of the number of hospitalizations due to diabetes across 2016-2022 for each county in California.
- __Revised Question__: Is there any county that had the highest mean diabetes-related hospitalization rate for more than three years between 2016 and 2022? The hospitalization rate refers to column `ObsRate_ICD10`, which is calculated by $\frac{\text{Count of Hospitalizations}}{\text{Population at risk}} \times 100,000$
- Justification: Our TA initially thought we were trying to solve the problem by creating visualizations. However, our original goal was to build a suitable model (e.g., a time series model). After analyzing the data, we realized that the limited number of time points made it difficult to construct a reliable model. Therefore, we shifted to compare hospitalization rates across counties for each year and investigate whether any county consistently had the highest rate over this period.

lsy notice: we are using 4 categories right now! short/long/uncontrolled/Composite!!!

## 3. Data Cleaning 
_(Our detailed data cleaning code and procedure are shown in cleaning/clean.ipynb on Github)_

There are two datasets we will be using throughout our research, and we have cleaned both of them before doing any combinings/joinings.

__Cleaning the first dataset__, `data-table.csv`, is fairly straightforward, since the dataset is already in good shape. The only steps we need to take are:
- Filter out the columns that we definitely won’t need; in this case, it will be `URL`.
- Only keep rows that satisfy our research criteria, meaning the data collected must be from California (`STATE` == 'CA') and obtained between 2016 and 2022 (2016 <= `YEAR` <= 2022).

__Cleaning the second dataset__, `rates-of-preventable-hospitalizations-for-selected-medical-conditions-by-county-.csv`, involves a bit more work. To start, we want to keep only the rows within our study range, which is from 2016 to 2022. Then, we filtered out some irrelevant columns like `Count_ICD9` and `Population_ICD9`, as these variables are based on older criteria before 2016 and contain only empty values after the new criteria were applied. Next, we need to exclude any hospitalization records caused by conditions unrelated to diabetes. In our case, there are only four diabetes-related conditions, so it’s actually quicker to use `isin` to include them, rather than excluding the others. Afterward, by using `info()`, we noticed an anomaly with a non-null value in one of the annotation columns. We looked into that specific observation and realized it was just a period showing in that column, so it wasn’t a major issue. As a second-to-last step, we further filtered out some columns that may not be helpful for our research questions, such as `RiskAdjRate_ICD10` and the annotation columns mentioned above. Finally, we used `shape`, `info()`, and `describe()` to ensure that our cleaned data is indeed in good format.

## 5. How EDA affects the analysis
EDA helps us better understand the data distribution and identify trends in hospitalization and death rates. Through EDA, we did not encounter missing or inconsistent data, so we do not need to perform transformations or imputations. Instead, we focus on analyzing the hospitalization rates, complications, and mortality without additional preprocessing, so actually EDA process does not affect much on the workflow we are going to carry out with our data.